In [33]:
#!/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
import numpy as np
import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
import itertools
import datetime
import sys


#from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "16g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

## SVD модель интернет-логов. Выделение тем. ##
Есть таблица интернет-логов  $m_{ij}$, каждая строка отвечает одному интернет-пользователю, каждый столбец - фрагменту урла. $m_{ij}=1$, если посетитель $i$ посетил фрагмент урла $j$ в рамках рассматриваемой сессии, иначе $m_{ij}=0$.

Выделим темы - группировки урлов и пользователей. Для этого воспользуемся модификацией SVD разложения. 

Зафиксируем количество тем $K$. Для каждого пользователя сформируем вектор длины $K$, отражающий степень близости темы к пользователю. Получаем матрицу $X_{ik}$. Индекс $i$ отвечает пользователю, $k$ отвечает теме. 

Аналогично сформируем вектор  длины $K$ для каждого фрагмента урла. Получаем матрицу $Y_{kj}$. $k$ отвечает теме, $j$ отвечает фрагменту урла.

Далее для каждого пользователя $i$ введем число $a_i$, которое в целом характеризует его интернет-активность.

Аналогично, элементы вектора $b_j$ отражают популярность урла $j$.

Параметр $\mu$ есть усредняющий скаляр по всей выборке.

Далее интерес пользователя  $i$ к фрагменту урла $j$ будем моделировать величиной $s_{ij} = \mu + a_i + b_j + \sum_k X_{ik} \cdot Y_{kj}$.

Для перевода этой величины в вероятность, воспользуемся логистической функцией: 
$\hat{m}_{ij} = 1/(1+e^{-s_{ij}})$. 

Таким образом, построенная величина $\hat{m}_{ij}$ есть  модельная оценка вероятности посещения пользователем $i$ фрагмента урла $j$.

Введем функцию ошибки $L = \sum_{ij} L_{ij} $,
$$L_{ij} = -m_{ij} \cdot \ln(\hat{m}_{ij}) - (1 - m_{ij}) \cdot \ln(1 - \hat{m}_{ij}) + \lambda_{0} \cdot \mu + \lambda_{1}\cdot (a_i^2 + b_j^2) + \lambda_{2} \cdot \sum_k(X_{ik}^2 + Y_{kj}^2)$$.

Для обучения будем использовать стохастический градиентный спуск (его модификацию Adadelta http://int8.io/comparison-of-optimization-techniques-stochastic-gradient-descent-momentum-adagrad-and-adadelta/#AdaDelta_8211_implementation).


In [73]:

start_time = datetime.datetime.now()
# Param selection
query = '''
select id_num,urlfr_num from 
(select id_num,urlfr_num,count(*) over (partition by id_num) as uf_cnt,count(*) over (partition by urlfr_num) as id_cnt from user_kposminin.visits_enum_20160412 
where id_num < 10000) a
where id_cnt > 10 and uf_cnt > 9
'''
# where uf_cnt > 9 and id_cnt > 500


K = 50
id_cnt = 10000
urlfr_cnt = 3000000
epochs = 15


#lmbd = np.array([0.01, 0.01, 0.01])

#step = np.array([0.001, 0.005, 0.01])

param_grid = []

 #   [[0.00001, 0.0001, 0.0001],[0.01, 0.05, 0.01]],
for base_step in np.exp(np.arange(-4,0,2)):
    for base_lmbd in np.exp(np.arange(-14,-8,2)):
        for var_step in np.array([[0.1,1,0.1],[0.01,1,10]]):
            for var_lmbd in np.array([[0.01,0.1,1],[1,0.1,0.01]]):
                param_grid.append([var_step * base_step,var_lmbd * base_lmbd])

'''
# Load and parse the data
sampled_data = hc.sql(query) \
            .collect()
print('Sampled data consists of {} rows, {} id and {} uf.'.format(len(sampled_data),len(set([e[0] for e in sampled_data])),len(set([e[1] for e in sampled_data]))))

train,test = [], []
for r in sampled_data:
    if(np.random.rand() < 0.1):
        test.append(r)
    else:
        train.append(r)
'''
#  
# prediction is m^hat_{ij} = 1/(1+exp(-s_{ij})) where s_{ij} = mu + a[i] + b[j] + sum_k (X[i,k] * Y[k,j])
# error function is L_{ij} = -m_{ij}*ln(m^hat_{ij}) - (1 - m_{ij})*ln(1 - m^hat_{ij}) + 
#                            lmbd[0] * mu + lmbd[1]*(a_i^2 + b_j^2) + lmbd[2]*sum_k(X_{ik}^2 + Y_{kj}^2)
f = open('data/collab_filter_SGD_param_tuning.csv','a+')
f.write('\n\n New calc at {}'.format(datetime.datetime.now()))
stat = []
prev_id = -1
uf_visited= []
neg_sig_share = 2
rho = 0.9
eps = 1e-6

for step,lmbd in [[[1,1,1],[2.48e-04,2.48e-04,2.48E-03]]]:    
    #Init matrices
    mu, mu_gr_sq, mu_d_sq = np.random.rand(1) - 0.5 - 2, np.zeros(1), np.zeros(1) # One-element array to be able to update inside a procedure
    a, a_gr_sq,   a_d_sq = np.random.rand(id_cnt) - 0.5, np.zeros(id_cnt), np.zeros(id_cnt)
    b, b_gr_sq,   b_d_sq = np.random.rand(urlfr_cnt) - 0.5, np.zeros(urlfr_cnt), np.zeros(urlfr_cnt)
    X, X_gr_sq,   X_d_sq = np.random.rand(id_cnt, K) - 0.5, np.zeros([id_cnt, K]), np.zeros([id_cnt, K])
    Y, Y_gr_sq,   Y_d_sq = np.random.rand(K,urlfr_cnt) - 0.5, np.zeros([K,urlfr_cnt]), np.zeros([K,urlfr_cnt])

    def make_step(i, j, v):
        s = mu[0] + a[i] + b[j] + X[i,:].dot(Y[:,j])
        pred = 1./(1+np.exp(-s))
        err = v - pred # r[2] - pred
        grad_mu = -err + lmbd[0] * mu
        grad_ai = -err + lmbd[1] * a[i]
        grad_bj = -err + lmbd[1] * b[j]
        grad_xi = -err * Y[:,j] + lmbd[2] * X[i,:]
        grad_yj = -err * X[i,:] + lmbd[2] * Y[:,j]
        
        # TODO Implement Adadelta SGD version. 
        #http://int8.io/comparison-of-optimization-techniques-stochastic-gradient-descent-momentum-adagrad-and-adadelta/#AdaDelta_8211_implementation
        # 
        mu_gr_sq[0] = rho * mu_gr_sq[0] + (1-rho) * (grad_mu[0] ** 2)
        a_gr_sq[i]  = rho * a_gr_sq[i] + (1-rho) * (grad_ai ** 2)
        b_gr_sq[j]  = rho * b_gr_sq[j] + (1-rho) * (grad_bj ** 2)
        X_gr_sq[i,:]  = rho * X_gr_sq[i,:] + (1-rho) * (grad_xi ** 2)
        Y_gr_sq[:,j]  = rho * Y_gr_sq[:,j] + (1-rho) * (grad_yj ** 2)
        
        dmu = -(mu_d_sq + eps) ** 0.5 / (mu_gr_sq + eps) ** 0.5 * grad_mu
        dai = -(a_d_sq[i] + eps) ** 0.5 / (a_gr_sq[i] + eps) ** 0.5 * grad_ai
        dbj = -(b_d_sq[j] + eps) ** 0.5 / (b_gr_sq[j] + eps) ** 0.5 * grad_bj
        dXi = -(X_d_sq[i,:]  + eps) ** 0.5 / (X_gr_sq[i,:]  + eps) ** 0.5 * grad_xi
        dYj = -(Y_d_sq[:,j]  + eps) ** 0.5 / (Y_gr_sq[:,j]  + eps) ** 0.5 * grad_yj
        
        mu_d_sq[0] = rho * mu_d_sq[0] + (1-rho) * (dmu[0] ** 2)
        a_d_sq[i]  = rho * a_d_sq[i] + (1-rho) * (dai ** 2)
        b_d_sq[j]  = rho * b_d_sq[j] + (1-rho) * (dbj ** 2)
        X_d_sq[i,:]  = rho * X_d_sq[i,:] + (1-rho) * (grad_xi ** 2)
        Y_d_sq[:,j]  = rho * Y_d_sq[:,j] + (1-rho) * (grad_yj ** 2)
        
        mu[0]+= dmu[0] /5
        a[i] += dai /5
        b[j] += dbj / 5
        X[i,:] = X[i,:] + dXi /5
        Y[:,j] = Y[:,j] + dYj / 5
        
        #mu[0]   += - grad_mu * step[0]
        #a[i] += - grad_ai * step[1]
        #b[j] += - grad_bj * step[1]
        #X[i,:] = X[i,:] - grad_xi * step[2]
        #Y[:,j] = Y[:,j] - grad_yj * step[2]
    
    for epoch in range(epochs):
        for r in train:
            i,j = r[:2]
            make_step(i, j, 1)
            if i == prev_id:
                uf_visited.append(j)
            else:
                # Add negative examples
                ni = np.random.randint(len(train) - len(uf_visited))
                neg, k = 0, 0
                while(neg < int(len(uf_visited) * neg_sig_share)):
                    if(not train[(ni + k) % len(train)][1] in uf_visited):                        
                        make_step(prev_id, train[(ni + k) % len(train)][1],0)
                        neg += 1
                    k +=1 
                prev_id = i
                uf_visited = [j]
        
        #calc err
        err1_p ,err2_p, err1_n, err2_n = 0, 0, 0, 0
        for r in test:
            i,j = r[:2]
            s = mu[0] + a[i] + b[j] + X[i,:].dot(Y[:,j])
            pred = 1/(1+np.exp(-s))
            err1_p += - 1 * np.log(pred) # r[2] - pred
            err2_p += - 1 * np.log(pred) + lmbd[0] * mu[0] + lmbd[1] * (a[i] ** 2 + b[j] ** 2) + lmbd[2] * (X[i,:].dot(X[i,:]) + Y[:,j].dot(Y[:,j]))
            
            if i == prev_id:
                uf_visited.append(j)
            else:
                # Add negative examples
                ni = np.random.randint(len(test) - len(uf_visited))
                neg, k = 0, 0
                while(neg < int(len(uf_visited) * neg_sig_share)):
                    if(not test[(ni + k) % len(test)][1] in uf_visited): 
                        i1, j1 = prev_id, train[(ni + k) % len(train)][1]
                        s = mu[0] + a[i1] + b[j1] + X[i1,:].dot(Y[:,j1])
                        pred = 1/(1+np.exp(-s))
                        err1_n += - 1 * np.log(1 - pred) # r[2] - pred
                        err2_n += - 1 * np.log(1 - pred) + lmbd[0] * mu[0] + lmbd[1] * (a[i1] ** 2 + b[j1] ** 2) + lmbd[2] * (X[i1,:].dot(X[i1,:]) + Y[:,j1].dot(Y[:,j1]))                        
                        neg += 1
                    k +=1 
                prev_id = i
                uf_visited = [j]
        err1_t = (err1_p + err1_n) / (len(test) * (neg_sig_share + 1))
        err2_t = (err2_p + err2_n) / (len(test) * (neg_sig_share + 1))
        err1_p /= float(len(test))
        err2_p /= float(len(test))
        err1_n /= len(test) * neg_sig_share
        err2_n /= len(test) * neg_sig_share        
        
        stat=('{} ' * 9).format(step,lmbd,epoch,err1_t, err1_p,err1_n,err2_t, err2_p,err2_n)
        f.write(stat + '\n')
        print(stat)
        
print('Finish. Work time {}.'.format(datetime.datetime.now()- start_time))
f.write('\nFinish at {}. Work time {}.\n\n\n'.format(datetime.datetime.now(),datetime.datetime.now() - start_time))
f.close()

[1, 1, 1] [0.000248, 0.000248, 0.00248] 0 0.809257903106 2.09594658865 0.165913560335 0.855935721474 2.13917406658 0.214316548923 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 1 0.788582837572 2.17859933153 0.0935745905955 0.839298914975 2.23017948441 0.143858630258 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 2 0.803490759358 2.27255746797 0.0689574050549 0.860066898123 2.33192521053 0.124137741918 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 3 0.821735427527 2.34781430068 0.0586959909485 0.882792995481 2.41246765699 0.117955664727 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 4 0.83228223851 2.38041119311 0.0582177612093 0.896357199357 2.44858208016 0.120244758959 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 5 0.860782205622 2.47431898286 0.0540138170042 0.926557901789 2.54423676933 0.11771846802 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 6 0.854591438852 2.46078771085 0.0514933028514 0.921777362337 2.53230250667 0.116514790171 
[1, 1, 1] [0.000248, 0.000248, 0.00248] 7 0.868387273247 2.5176026912 0.0437795

{4}

In [12]:
mu = np.random.rand(1) - 0.5 - 6
def sd(a):
    print(mu[0])
    mu = mu + 3
    print(mu[0])

sd(2)
print('outside '+ str(mu[0]))

UnboundLocalError: local variable 'mu' referenced before assignment

In [25]:
i= 1
j = 1
v = 1
if True:
    mu, mu_gr_sq, mu_d_sq = np.random.rand(1) - 0.5 -6, np.zeros(1), np.zeros(1) # One-element array to be able to update inside a procedure
    a, a_gr_sq,   a_d_sq = np.random.rand(id_cnt) - 0.5, np.zeros(id_cnt), np.zeros(id_cnt)
    b, b_gr_sq,   b_d_sq = np.random.rand(urlfr_cnt) - 0.5, np.zeros(urlfr_cnt), np.zeros(urlfr_cnt)
    X, X_gr_sq,   X_d_sq = np.random.rand(id_cnt, K) - 0.5, np.zeros([id_cnt, K]), np.zeros([id_cnt, K])
    Y, Y_gr_sq,   Y_d_sq = np.random.rand(K,urlfr_cnt) - 0.5, np.zeros([K,urlfr_cnt]), np.zeros([K,urlfr_cnt])
    if True:
        s = mu[0] + a[i] + b[j] + X[i,:].dot(Y[:,j])
        pred = 1./(1+np.exp(-s))
        err = v - pred # r[2] - pred
        grad_mu = -err + lmbd[0] * mu
        grad_ai = -err + lmbd[1] * a[i]
        grad_bj = -err + lmbd[1] * b[j]
        grad_xi = -err * Y[:,j] + lmbd[2] * X[i,:]
        grad_yj = -err * X[i,:] + lmbd[2] * Y[:,j]
        
        # TODO Implement Adadelta SGD version. 
        #http://int8.io/comparison-of-optimization-techniques-stochastic-gradient-descent-momentum-adagrad-and-adadelta/#AdaDelta_8211_implementation
        # 
        mu_gr_sq[0] = rho * mu_gr_sq[0] + (1-rho) * (grad_mu[0] ** 2)
        a_gr_sq[i]  = rho * a_gr_sq[i] + (1-rho) * (grad_ai ** 2)
        b_gr_sq[j]  = rho * b_gr_sq[j] + (1-rho) * (grad_bj ** 2)
        X_gr_sq[i,:]  = rho * X_gr_sq[i,:] + (1-rho) * (grad_xi ** 2)
        Y_gr_sq[:,j]  = rho * Y_gr_sq[:,j] + (1-rho) * (grad_yj ** 2)
        
        dmu = -(mu_d_sq + eps) ** 0.5 / (mu_gr_sq + eps) ** 0.5 * grad_mu
        dai = -(a_d_sq[i] + eps) ** 0.5 / (a_gr_sq[i] + eps) ** 0.5 * grad_ai
        dbj = -(b_d_sq[j] + eps) ** 0.5 / (b_gr_sq[j] + eps) ** 0.5 * grad_bj
        dXi = -(X_d_sq[i,:]  + eps) ** 0.5 / (X_gr_sq[i,:]  + eps) ** 0.5 * grad_xi
        dYj = -(Y_d_sq[:,j]  + eps) ** 0.5 / (Y_gr_sq[:,j]  + eps) ** 0.5 * grad_yj
        
        mu_d_sq[0] = rho * mu_d_sq[0] + (1-rho) * (dmu[0] ** 2)
        a_d_sq[i]  = rho * a_d_sq[i] + (1-rho) * (dai ** 2)
        b_d_sq[j]  = rho * b_d_sq[j] + (1-rho) * (dbj ** 2)
        X_d_sq[i,:]  = rho * X_d_sq[i,:] + (1-rho) * (grad_xi ** 2)
        Y_d_sq[:,j]  = rho * Y_d_sq[:,j] + (1-rho) * (grad_yj ** 2)
        
        mu[0]+= dmu
        a[i] += dai
        b[j] += dbj
        X[i,:] = X[i,:] + dXi
        Y[:,j] = Y[:,j] + dYj

In [26]:
import numpy as np
K = 50
id_cnt = 10000
urlfr_cnt = 3000000
epochs = 15
step,lmbd = [[[1,1,1],[0.0001, 0.0001, 0.0001]]][0]
rho = 1e-6
stat = []
prev_id = -1
uf_visited= []
neg_sig_share = 2
rho = 0.9
eps = 1e-6

In [27]:
mu, mu_gr_sq, mu_d_sq = np.random.rand(1) - 0.5 -6, np.zeros(1), np.zeros(1)

In [28]:
mu_gr_sq[0] = rho * mu_gr_sq[0] + (1-rho) * (grad_mu[0] ** 2)

In [29]:
dmu = -(mu_d_sq + eps) ** 0.5 / (mu_gr_sq + eps) ** 0.5 * grad_mu
dmu

array([ 0.00316226])

In [30]:
grad_mu

array([-0.99967312])

In [31]:
mu

array([-5.70989463])